In [24]:
import pandas as pd
import numpy as np

In [25]:
#null handling
df = pd.read_csv("afl_game_by_game_results_1965_2022.csv")
df.isnull().sum()

#replace with 0
df['behinds']

0         21-8
1         16-9
2         25-9
3        11-13
4          8-8
         ...  
18521     8-10
18522     15-4
18523     12-8
18524      8-7
18525    11-13
Name: behinds, Length: 18526, dtype: object

In [26]:
stats_columns = ['kicks', 'marks', 'handballs', 'disposals', 'goals', 'behinds', 'hit_outs', 'tackles', 'rebound_50s', 
                 'inside_50s', 'clearances', 'clangers', 'freekicks_for', 'freekicks_agains', 'brownlow_votes', 
                 'contested_possesions', 'uncontested_possesions', 'contested_marks', 'marks_inside_50', 'one_percenters',
                 'bounces', 'goal_assist']




In [27]:
#splitting the stats to 2 teams
for col in stats_columns:
    #using regexp to filter off all incorrect formats for stats
    df[col] = df[col].where(df[col].str.fullmatch(r'^\d+-\d+$', na=False), np.nan)
    
    # Split the stats into two columns for each team (team1_stats, team2_stats)
    df[[f'{col}_team1', f'{col}_team2']] = df[col].str.split('-',expand=True)
    
    # Convert these columns to numeric values
    df[f'{col}_team1'] = pd.to_numeric(df[f'{col}_team1'], errors='coerce')
    df[f'{col}_team2'] = pd.to_numeric(df[f'{col}_team2'], errors='coerce')

#Calculate the total score for each team: goals * 6 + behinds
df['score_team1'] = df['goals_team1'] * 6 + df['behinds_team1']
df['score_team2'] = df['goals_team2'] * 6 + df['behinds_team2']

#dropping all original stat columns 
df = df.drop(columns= stats_columns)


In [28]:
#creating match_id with year, round, teams playing
df['match_id'] = df.apply(lambda row: tuple([row['year'], row['round'], sorted([row['team'], row['Opponent']])]), axis=1)

#drop duplicates
df = df.drop_duplicates(subset = 'match_id', keep= 'first')


In [29]:
df

,Unnamed: 0.1,Unnamed: 0,team,round,Opponent,year,kicks_team1,kicks_team2,marks_team1,marks_team2,...,marks_inside_50_team2,one_percenters_team1,one_percenters_team2,bounces_team1,bounces_team2,goal_assist_team1,goal_assist_team2,score_team1,score_team2,match_id
0,0,0,Carlton,R1,Geelong,1977,228.0,197.0,83.0,59.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,123.0,44.0,"(1977, R1, [Carlton, Geelong])"
1,1,1,Carlton,R2,Fitzroy,1977,214.0,227.0,67.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.0,81.0,"(1977, R2, [Carlton, Fitzroy])"
2,2,2,Carlton,R3,St Kilda,1977,230.0,180.0,86.0,65.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,169.0,63.0,"(1977, R3, [Carlton, St Kilda])"
3,3,3,Carlton,R4,Hawthorn,1977,221.0,214.0,65.0,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83.0,91.0,"(1977, R4, [Carlton, Hawthorn])"
4,4,4,Carlton,R5,South Melbourne,1977,214.0,190.0,78.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,134.0,104.0,"(1977, R5, [Carlton, South Melbourne])"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18446,264,13,St Kilda,R15,West Coast,1991,189.0,215.0,76.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.0,135.0,"(1991, R15, [St Kilda, West Coast])"
18454,272,21,St Kilda,R24,Sydney,1991,193.0,187.0,77.0,50.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,156.0,116.0,"(1991, R24, [St Kilda, Sydney])"
18464,282,8,Sydney,R10,West Coast,1991,185.0,211.0,47.0,72.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.0,105.0,"(1991, R10, [Sydney, West Coast])"
18466,284,10,Sydney,R12,Footscray,1991,239.0,229.0,43.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0,33.0,"(1991, R12, [Footscray, Sydney])"


In [30]:
#dropping unused columns
df.drop(columns= ['Unnamed: 0.1', 'Unnamed: 0', 'match_id'], inplace= True)
df = df.sort_values(by=['year', 'round', 'team'])
df.columns = df.columns.str.strip()
df = df.rename(columns={'team': 'team1', 'Opponent': 'team2'})

In [32]:
#export
df.to_csv('cleaned_kaggle.csv')